# Data Science Bowl 2018

Find cell nuclei in microscope images

https://www.kaggle.com/c/data-science-bowl-2018/data

In [12]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Testing IOU


In [50]:
from evaluations.iou import log_iou, iou_loss
import plot_helpers.plot as plot
import data_loaders

import numpy as np

In [19]:
summary_masks_raw, weights_raw = data_loaders.masks_and_weights("saved_weights")

In [54]:
offset = 1
true = summary_masks_raw[0]
pred = true[offset:, offset:]
pred = np.pad(pred, ((0, offset), (0, offset)), mode="constant")

In [44]:
"""
plot.imshow(true)
plot.imshow(pred)
"""

'\nplot.imshow(true)\nplot.imshow(pred)\n'

In [58]:
log_iou(pred, true)
print(iou_loss(pred, true))

Thresh	TP	FP	FN	Prec.
0.500	9	0	0	1.000
0.550	9	0	0	1.000
0.600	8	1	1	0.800
0.650	8	1	1	0.800
0.700	8	1	1	0.800
0.750	8	1	1	0.800
0.800	4	5	5	0.286
0.850	1	8	8	0.059
0.900	0	9	9	0.000
0.950	0	9	9	0.000
AP	-	-	-	0.554
0.554453781512605
